# Document用jsonファイルの作成

In [9]:
import pandas as pd
import numpy as np
import glob
import json
import datetime
import shutil
# 初期値の設定
date_str=datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
df_format = pd.DataFrame({'title':[],"md_path": [],"language":[], "date": []})

In [10]:
df = pd.read_csv("./data/md_data.csv",index_col=0)
df

,title,md_path,language,date
1,DataFrameのforループ処理,./data/md_file/doc001.md,python,2025-01-01 09:05:43


## 追加マークダウンファイルのインポート

In [11]:
df = pd.read_csv("./data/md_data.csv",index_col=0)
markdown_files = glob.glob("./data/add_md/*.md")
len(markdown_files)
## markdown_filesがない場合は処理を終了する
if len(markdown_files)==0:
  print("markdown_files is not found")
else:
  ## markdown_filesがある場合
  for md_path in markdown_files:
    date_str=datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    # markdownファイルのタイトルをmdファイルの２行目の文字列から取得
    with open(md_path) as f:
      lines = f.readlines()
      title = lines[1].replace("# ","").replace("\n","")
      language = lines[3].replace("# ","").replace("\n","")
    #title = md_path.split("/")[-1].replace(".md","")
    df_2 = pd.DataFrame([{'title':title,"md_path": md_path.replace('add_md','md_file'),"language":language, "date": date_str}])
    df = pd.concat([df, df_2], axis=0)
    ## 追加のファイルを移動する
    md_path_new = md_path.replace('add_md','md_file')
    !mv {md_path} {md_path_new}
  # インデックスを1から通し番号に設定
  df.index = range(1, len(df) + 1)
  df.to_csv("./data/md_data.csv", index=True)
# imgファイルを移動
img_files = glob.glob("./data/add_md/img/*")
if len(img_files)==0:
  print("not found img files")
else:
  for img_file in img_files:
    img_file_name = f'../public/img/{img_file.split("/")[-1]}'
    shutil.move(img_file, img_file_name)
## 
df


not found img files


,title,md_path,language,date
1,DataFrameのforループ処理,./data/md_file/doc001.md,python,2025-01-01 09:05:43
2,pandasで重複を削除,./data/md_file/doc002.md,- python,2025-01-01 09:53:34


## jsonファイルの作成

In [12]:
df2 =  pd.read_csv("./data/md_data.csv",index_col=0)
df2['date'] = pd.to_datetime(df2['date'])
#df2.sort_values("date",ascending=False,inplace=True)
json_list=[]
for index, row in df2.iterrows():
  with open(row['md_path'], "r", encoding="utf-8") as file:
    content = file.read()
    content = content.replace("](./img","](/img")
  json_item = {
    "id": index,
    "title": row['title'],
    "language": row['language'],
    "content": content,
    "updatedAt": row['date'].strftime('%Y-%m-%d %H:%M:%S')
  }
  json_list.append(json_item)
## jsonファイルに書き換える
with open('../public/data/document.json', 'w') as f:
    json.dump(json_list, f,ensure_ascii=False, indent=4)
df2

,title,md_path,language,date
1,DataFrameのforループ処理,./data/md_file/doc001.md,python,2025-01-01 09:05:43
2,pandasで重複を削除,./data/md_file/doc002.md,python,2025-01-01 09:53:34
